In [1]:
import numpy as np
from scipy.integrate import quad
from time import time

# import functions
import function as ade

In [2]:
# Setup
n = 300
M = np.array([n, 2 * n])  # Sample sizes
S = 500  # Simulation size
T = lambda th, x: th + x  # Transformation map
g = np.array([20, 250])  # Neural network configuration
node = 5  # Number of nodes
th0 = 0  # True parameter


# Check size consistency
assert len(M) == len(g), "Size of M and g must be the same."

# Analytic calculation
log_p0 = lambda x: ade.loglogisticpdf(x, th0, 1)
log_p = lambda th, x: ade.loglogisticpdf(x, th, 1)

ld = lambda th, x: -1 + 2 * ade.logisticcdf(x - th, 0, 1)
ldd = lambda th, x: -2 * ade.logisticcdf(x - th, 0, 1) * ade.logisticcdf(-(x - th), 0, 1)

log_D = lambda th, x: log_p0(x) - ade.logaddexp(log_p0(x), log_p(th, x))
log_UmD = lambda th, x: log_p(th, x) - ade.logaddexp(log_p0(x), log_p(th, x))
p0 = lambda x: np.exp(log_p0(x))
p = lambda th, x: np.exp(log_p(th, x))
D = lambda th, x: np.exp(log_D(th, x))

In [3]:
# Precompute constants for integration
meat_func = lambda x: 4 * np.exp(log_D(th0, x) + log_UmD(th0, x)) * ld(th0, x)**2

In [4]:
Meat_1, error1 = quad(lambda x: meat_func(x) * p(th0, x), -500, 500, epsabs=1e-12)
Meat_2, error2 = quad(lambda x: meat_func(x) * p0(x), -500, 500, epsabs=1e-12)

In [5]:
V = Meat_1 + (n / M) * Meat_2

In [6]:
bun_func = lambda x: 2 * (D(th0, x) * ld(th0, x)**2 + (ldd(th0, x) + ld(th0, x)**2) * log_UmD(th0, x))

In [7]:
I_tilde = quad(lambda x: bun_func(x) * p(th0, x), -500, 500, epsabs=1e-20)[0]

In [8]:
# Asymptotic variances
V_Adv = V/I_tilde**2
V_MLE = -1 / quad(lambda x: ldd(th0, x) * p0(x), -500, 500, epsabs=1e-20)[0]

In [9]:
# Orthogonality
K = 50
th_grid = np.linspace(-0.7, 0.7, K)

In [10]:
# Generate real and latent data
X = ade.logisticrnd(0, 1, (n, 1))
Z = ade.logisticrnd(0, 1, (max(M), 1))

# Precompute Z slices
Z_slices = {m: Z[:m] for m in M}

# Initialize arrays for results
LL_grid = -np.mean([log_p(th, X) for th in th_grid], axis=1)
oD_grid = np.zeros((len(M), K))
cD_grid = np.zeros((len(M), K))
NND_grid = np.zeros((len(M), K))

In [54]:
# Vectorized computation over th_grid and M
start_time = time()
for m_idx, m_size in enumerate(M):
    Z_m = Z_slices[m_size]
    T_Z = np.array([T(th, Z_m) for th in th_grid])  # Ensure shape consistency

    # Compute grids and flatten results to match the expected shape
    oD_grid[m_idx, :] = np.mean(
        [log_D(th, X) + log_UmD(th, T_Z[i]) for i, th in enumerate(th_grid)],
        axis=1
    ).reshape(-1)

    cD_grid[m_idx, :] = np.mean(
        [ade.loss(X, T_Z[i])[0] for i in range(K)]
    ).reshape(-1)

    # NND_grid[m_idx, :] = np.mean(
    #     [ade.NND(X.T, T_Z[i].T, g[m_idx], node) for i in range(K)]
        
    # ).reshape(-1)
end_time = time()

ValueError: operands could not be broadcast together with shapes (300,1) (600,1) 

In [50]:
# Print elapsed time
print(f"Optimized elapsed time: {end_time - start_time:.2f} seconds")

NameError: name 'end_time' is not defined

In [12]:
for m_idx, m_size in enumerate(M):
    Z_m = Z_slices[m_size]
    T_Z = np.array([T(th, Z_m) for th in th_grid])  # Ensure shape consistency
    for i, th in enumerate(th_grid):
        print(f"Shape of log_D({th}, X):", log_D(th, X).shape)
        print(f"Shape of log_UmD({th}, T_Z[{i}]):", log_UmD(th, T_Z[i]).shape)

Shape of log_D(-0.7, X): (300, 1)
Shape of log_UmD(-0.7, T_Z[0]): (300, 1)
Shape of log_D(-0.6714285714285714, X): (300, 1)
Shape of log_UmD(-0.6714285714285714, T_Z[1]): (300, 1)
Shape of log_D(-0.6428571428571428, X): (300, 1)
Shape of log_UmD(-0.6428571428571428, T_Z[2]): (300, 1)
Shape of log_D(-0.6142857142857142, X): (300, 1)
Shape of log_UmD(-0.6142857142857142, T_Z[3]): (300, 1)
Shape of log_D(-0.5857142857142856, X): (300, 1)
Shape of log_UmD(-0.5857142857142856, T_Z[4]): (300, 1)
Shape of log_D(-0.5571428571428572, X): (300, 1)
Shape of log_UmD(-0.5571428571428572, T_Z[5]): (300, 1)
Shape of log_D(-0.5285714285714285, X): (300, 1)
Shape of log_UmD(-0.5285714285714285, T_Z[6]): (300, 1)
Shape of log_D(-0.5, X): (300, 1)
Shape of log_UmD(-0.5, T_Z[7]): (300, 1)
Shape of log_D(-0.4714285714285714, X): (300, 1)
Shape of log_UmD(-0.4714285714285714, T_Z[8]): (300, 1)
Shape of log_D(-0.44285714285714284, X): (300, 1)
Shape of log_UmD(-0.44285714285714284, T_Z[9]): (300, 1)
Shape of